# DenseNet baseline test

In [1]:
import numpy as np
import pandas as pd
import h5py
import torch
import cv2
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import os
from torch.utils.data import Dataset
from torchvision import transforms, models
from torchvision.models import densenet121, DenseNet121_Weights
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from dataset import HDF5Dataset

## Preprocess Data

In [2]:
# Load metadata
metadata = pd.read_csv("../train-metadata.csv")
hdf5 = h5py.File("../train-image.hdf5", "r")

# Sample balanced malignant and benign
malignant_ids = metadata[metadata["target"] == 1].sample(n=393, random_state=42)["isic_id"].tolist()
benign_ids = metadata[metadata["target"] == 0].sample(n=393, random_state=42)["isic_id"].tolist()

def load_images(ids):
    images = []
    labels = []
    for image_id in ids:
        image = hdf5[image_id][()]
        image = np.frombuffer(image, dtype=np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (128, 128))
        images.append(image)
        labels.append(1 if image_id in malignant_ids else 0)
    return images, labels

images_mal, labels_mal = load_images(malignant_ids)
images_ben, labels_ben = load_images(benign_ids)

# Combine and split
all_images = np.array(images_mal + images_ben)
all_labels = np.array(labels_mal + labels_ben)

X_train, X_val, y_train, y_val = train_test_split(all_images, all_labels, test_size=0.1, stratify=all_labels, random_state=42)

/var/folders/4w/dhmdb97x63lfq799n_46pwc80000gn/T/ipykernel_22711/2231450438.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv("../train-metadata.csv")


## Train

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
weights = DenseNet121_Weights.DEFAULT
transform = weights.transforms()
model = densenet121(weights=weights)
model.classifier = nn.Linear(model.classifier.in_features, 1)
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [5]:
train_dataset = HDF5Dataset(X_train, y_train, augment=True, transform=transform)
val_dataset = HDF5Dataset(X_val, y_val, augment=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=1)

In [6]:
num_epochs = 20
train_losses, val_losses = [], []
val_accuracies = []
val_precisions = []
val_recalls = []
val_f1s = []
# partial AUC, used to compare between models 
# v_gt = abs(np.asarray(Y_test) - 1)
# v_pred = np.array([1.0 - x for x in Y_probs])
# max_fpr = abs(1 - min_tpr)
# partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=0.8)
# partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs = inputs.to(device)
        labels = labels.float().unsqueeze(1).to(device)  # [B, 1]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    train_losses.append(epoch_loss / len(train_loader))

    # === Validation ===
    model.eval()
    val_loss = 0
    preds, targets = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds.extend(torch.sigmoid(outputs).cpu().numpy())
            targets.extend(labels.cpu().numpy())

    val_losses.append(val_loss / len(val_loader))
    val_auc = roc_auc_score(targets, preds)
    
    # Threshold predictions at 0.5
    pred_labels = (np.array(preds) >= 0.5).astype(int)
    true_labels = np.array(targets).astype(int)

    val_acc = accuracy_score(true_labels, pred_labels)
    val_precision = precision_score(true_labels, pred_labels)
    val_recall = recall_score(true_labels, pred_labels)
    val_f1 = f1_score(true_labels, pred_labels)

    print(f"Epoch {epoch+1}: Acc = {val_acc:.4f}, Precision = {val_precision:.4f}, Recall = {val_recall:.4f}, F1 = {val_f1:.4f}") 
    val_accuracies.append(val_acc)
    val_precisions.append(val_precision)
    val_recalls.append(val_recall)
    val_f1s.append(val_f1)
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_losses[-1],
        'val_loss': val_losses[-1],
        'val_auc': val_auc
    }, f"checkpoints/DenseNet_epoch_{epoch+1}.pth")

In [ ]:
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("DenseNet Training Loss")
plt.legend()
plt.savefig("DenseNet_loss_curve.png")